# On Classifier Evaluation

In [1]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Confusion Matrix

For classification problems, the target is a categorical variable. This means that we can simply count the number of times that our model predicts the correct category and the number of times that it predicts something else.

We can visualize this by means of a **confusion matrix**, which displays counts of correct and incorrect predictions. We'll explore this below. There are [many ways](https://docs.google.com/document/d/12BkMOXt5-iMzw_y_zFLZNkPlgAy1rj1t9gJ6LXi2bqQ/edit?usp=sharing) of evaluating a classification model, but most derive from the confusion matrix.

## Lottery Number Prediction

Suppose I want to train a model to predict whether a string of six numbers (a "ticket") would win the lottery or not. What sort of data might I use?

### Scenario 1

I gather all the winning tickets from the last 10000 days or so. So I have one column for the tickets themselves, and a Boolean column indicating whether the ticket was a winner or not.

Now if all the tickets on which my model trains are *winning* tickets, then it would predict every ticket to win! Suppose I test the model on a set of 1000 tickets, and suppose that there is exactly one winning ticket among those 1000. My model will always predict the ticket to win. Let's think about what the confusion matrix will look like.

In [2]:
# Setting up the true values

y_true = np.zeros(1000)
y_true[500] = 1

y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [3]:
# Setting up the predictions

y_pred = np.ones(1000)

y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [4]:
# Defining the confusion matrix

cm_1 = confusion_matrix(y_true, y_pred)

The confusion matrix should tell us that we have 999 false positives (999 losing tickets predicted to win) and 1 true positive (1 winning ticket predicted to win):

In [5]:
cm_1

array([[  0, 999],
       [  0,   1]])

Notice the way that sklearn displays its confusion matrix: The rows are \['actually false', 'actually true'\]; the columns are \['predicted false', 'predicted true'\].

So it displays:

$\begin{bmatrix}
TN & FP \\
FN & TP
\end{bmatrix}$

In [6]:
tn = cm_1[0, 0]
fp = cm_1[0, 1]
fn = cm_1[1, 0]
tp = cm_1[1, 1]

Let's see if we can calculate some of our metrics for this matrix.

**Accuracy** = $\frac{TP + TN}{TP + TN + FP + FN}$

In words: How often did my model get the right answer?

In [7]:
# Accuracy

print(f'Accuracy: ({tp} + {tn}) / ({tp} + {tn} + {fp} + {fn}) = 1/1000 = {1/1000}')

Accuracy: (1 + 0) / (1 + 0 + 999 + 0) = 1/1000 = 0.001


**Recall** = $\frac{TP}{TP + FN}$

In words: How often did my model correctly predict winning tickets?

In [8]:
# Recall

print(f'Recall: {tp} / ({tp} + {fn}) = 1/1 = {1}')

Recall: 1 / (1 + 0) = 1/1 = 1


**Precision** = $\frac{TP}{TP + FP}$

In words: How often was my model's prediction of 'winner' correct?

In [9]:
# Precision

print(f'Precision: {tp} / ({tp} + {fp}) = 1 / 1000 = {1/1000}')

Precision: 1 / (1 + 999) = 1 / 1000 = 0.001


**F-1 Score** = $\frac{2PrRc}{Pr + Rc}$ = $\frac{2TP}{2TP + FP + FN}$

In [ ]:
# F-1 Score

print(f'F-1 Score: (2)({tp}) / ((2)({tp}) + {fp} + {fn}) = 2 / 1001 = {round(2/1001, 3)}')

### Scenario 2

Well, my recall was good, but everything else I measured was terrible! Can I do better?

This time I'll train my model in a much different way. I'll give it the tickets of 10000 people who played the lottery yesterday. Suppose that there are one winning ticket and 9999 losers. Now I test the model on the same 1000 tickets from before in Scenario 1.

This time my model will almost always predict a ticket to lose. Suppose that, in the 1000 predictions, it makes only one prediction of a winner, and suppose that this prediction is wrong.

#### Exercise: Set up the confusion matrix and compute our metrics for this new scenario.

In [10]:
y_true2 = np.zeros(1000)
y_true2[0] = 1

y_preds2 = np.zeros(1000)
y_preds2[1] = 1

In [11]:
cm2 = confusion_matrix(y_true2, y_preds2)

In [12]:
cm2

array([[998,   1],
       [  1,   0]])

In [14]:
tn = cm2[0, 0]
fp = cm2[0, 1]
fn = cm2[1, 0]
tp = cm2[1, 1]

In [18]:
# Accuracy

print(f'Accuracy: ({tp} + {tn}) / ({tp} + {tn} + {fp} + {fn}) = {tp+tn}/{tp+tn+fp+fn} = {(tp+tn) / (tp+tn+fp+fn)}')

Accuracy: (0 + 998) / (0 + 998 + 1 + 1) = 998/1000 = 0.998


In [19]:
import sklearn.metrics

In [20]:
sklearn.metrics.accuracy_score(y_true2, y_preds2)

0.998

In [22]:
sklearn.metrics.recall_score(y_true2, y_preds2)

0.0

In [25]:
sklearn.metrics.precision_score(y_true2, y_preds2)

0.0

### Lessons

The last classifier had a really high accuracy, but everything else was terrible.

It occurs to me now that if I'm really going to get a good model running, I'm going to have to have good numbers of **both winning and losing  tickets**. The first model didn't see enough losers, and the second model didn't see enough winners.

Of course, I could just be more careful about my data collection, but there are often easier fixes. One of the most effective strategies is **oversampling the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html).

## Breast Cancer Prediction

Let's see what results we get on scikit-learn's breast cancer dataset:

### Loading the Data

In [26]:
preds, target = load_breast_cancer(return_X_y=True)

In [28]:
data = load_breast_cancer()

In [29]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

One way to measure the relative size of our classes:

In [27]:
sum(target) / len(target)

0.6274165202108963

Let's assume that this is close enough to "balanced"!

### Splitting and Scaling:

In [32]:
X_train, X_test, y_train, y_test = train_test_split(preds, target,
                                                   random_state=42)

In [33]:
ss = StandardScaler()

ss.fit(X_train)

X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

### Model Fitting

In [34]:
logreg = LogisticRegression(solver='lbfgs', max_iter=10000,
                           random_state=42)

logreg.fit(X_train_sc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Confusion Matrix

In [36]:
cm_bc = confusion_matrix(y_test, logreg.predict(X_test_sc))

In [40]:
sklearn.metrics.accuracy_score(y_test, logreg.predict(X_test_sc))

0.9790209790209791

In [41]:
sklearn.metrics.recall_score(y_test, logreg.predict(X_test_sc))

0.9775280898876404

In [42]:
sklearn.metrics.precision_score(y_test, logreg.predict(X_test_sc))

0.9886363636363636

### Interpretation

#### Exercise: What are the precision, recall, and accuracy of our model?

## Which Metric Should I Care About?

Well, it depends.

### General Lessons

First, let's make some general observations about the metrics we've so far defined.

Accuracy:
- Pro: Takes into account both false positives and false negatives.
- Con: Can be misleadingly high when there is a significant class imbalance. (A lottery-ticket predictor that *always* predicts a loser will be highly accurate.)

Recall:
- Pro: Highly sensitive to false negatives.
- Con: No sensitivity to false positives.

Precision:
- Pro: Highly sensitive to false positives.
- Con: No sensitivity to false negatives.

F-1 Score:
- Harmonic mean of recall and precision.

### Cost

The nature of your business problem will help you determine which metric matters.

Sometimes false positives are much worse than false negatives: Arguably, a model that compares a sample of crime-scene DNA with the DNA in a city's database of its citizens presents one such case. Here a false positive would mean falsely identifying someone as having been present at a crime scene, whereas a false negative would mean only that we fail to identify someone who really was present at the crime scene as such.

On the other hand, consider a model that inputs X-ray images and predicts the presence of cancer. Here false negatives are surely worse than false positives: A false positive means only that someone without cancer is misdiagnosed as having it, while a false negative means that someone with cancer is misdiagnosed as *not* having it.

### Strategies

There are a couple of strategies for trying to quantify these important differences in value between false positives and false negatives.

#### Strategy 1: Use a cost matrix.

One might assign different weights to the costs associated with false positives and false negatives. (We'll standardly assume that the costs associated with *true* positives and negatives are negligible.)

**Example**. Suppose we are in the DNA prediction scenario above. Then we might construct the following cost matrix:

In [43]:
cost = np.array([[0, 10], [3, 0]])
cost

array([[ 0, 10],
       [ 3,  0]])

This cost matrix will allow us to compare models if we have access to those models' rates of false positives and false negatives, i.e. if we have access to the models' confusion matrices!

**Problem**. Given the cost matrix above and the confusion matrices below, which model should we go with?

In [44]:
conf1, conf2 = np.array([[100, 10], [30, 300]]), np.array([[120, 20], [0, 300]])

print(conf1, 2*'\n', conf2)

[[100  10]
 [ 30 300]] 

 [[120  20]
 [  0 300]]


#### Strategy 2: Adjust the thresholds for false positives and false negatives.

Let's turn to this now!

In [45]:
def cost_calc(cost, cm):
    return cost[0, 1] * cm[0, 1] + cost[1, 0] * cm[1, 0]

# for model 1:
cost1 = cost_calc(cost = cost, cm=conf1)

# for model 2:
cost2 = cost_calc(cost = cost, cm = conf2)

print(cost1, cost2)

190 200
